### taeyoonahn

**1.유니콘기업과 연금보험료의 상관관계에 대한 가정들**   
**&nbsp;&nbsp;1.1. 연금보험료가 높으면 해당 기업은 유니콘 기업일 것이다.   
&nbsp;&nbsp;1.2. 연금보험료 상승률이 높으면 해당 기업은 유니콘 기업일 것이다.   
&nbsp;&nbsp;1.3. 매출/연금보험료가 높으면 해당 기업은 유니콘 기업일 것이다.   
&nbsp;&nbsp;1.4. 매출/연금보험료의 상승률이 높으면 해당 기업은 유니콘 기업일 것이다.**

**2. 연금보험료 측면 데이터 분석**   
**&nbsp;&nbsp;2.1. 기간 내 연금보험료 평균값이 높은 기업 분석       
&nbsp;&nbsp;2.2. 기간 내 연금보험료 평균 상승율이 높은 기업 분석   
&nbsp;&nbsp;2.3. 기간 내 매출/연금보험료 평균값이 높은 기업 분석   
&nbsp;&nbsp;2.4. 기간 내 매출/연금보험료 상승률이 높은 기업 분석**

**3. 최종산출물**   
**&nbsp;&nbsp;3.1. 기간 내 연금보험료 평균값이 높은 기업 Top 50개  
&nbsp;&nbsp;3.2. 기간 내 연금보험료 평균 상승율이 높은 기업 Top 50개    
&nbsp;&nbsp;3.3. 기간 내 매출/연금보험료 평균값이 높은 기업 Top 50개    
&nbsp;&nbsp;3.4. 기간 내 매출/연금보험료 상승률이 높은 기업 Top 50개**

In [72]:
#package 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font", family="Malgun Gothic")

from datetime import datetime
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings(action='ignore')

In [73]:
#실수는 소수점 두자리까지 표기
pd.options.display.float_format = '{:.2f}'.format

In [74]:
#데이터 불러오기
df_ori=pd.read_csv('company_nps_data.csv',engine = "python", encoding = "CP949")
df_ori

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
0,233757,41688077,2018,7,8912380,36
1,123464,1760695,2018,3,942380,5
2,342124,3221341,2018,7,781180,4
3,386420,4815584,2015,11,3795900,14
4,129243,9799736,2018,10,40724680,151
...,...,...,...,...,...,...
99796,171439,4630622,2017,2,2402740,11
99797,239388,1045021,2018,10,1428800,9
99798,272304,181898,2018,3,392560,4
99799,362194,2100218,2018,12,874780,3


In [75]:
#데이터 정보 확인
print(df_ori.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99801 entries, 0 to 99800
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   회사ID      99801 non-null  int64
 1   연매출액      99801 non-null  int64
 2   년도        99801 non-null  int64
 3   월         99801 non-null  int64
 4   월별_연금보험료  99801 non-null  int64
 5   월별_직원수    99801 non-null  int64
dtypes: int64(6)
memory usage: 4.6 MB
None


> **국민연금보험료
국민연금보험료란 국민연금 가입자들이 나중에 연금을 타기 위해 매월 내는 돈을 말한다.   
현재 보험료율은 9%로, 가입자는 매월 소득 중 9%를 보험료로 납부하고 있다.   
직장에 다니는 사람은 회사에서 절반을 내줘 실질적으로는 4.5%를 낸다.(출처 : 네이버지식백과)**  

In [76]:
#데이터 결측치 확인
df_ori.isnull()
df_ori.isnull().sum()

회사ID        0
연매출액        0
년도          0
월           0
월별_연금보험료    0
월별_직원수      0
dtype: int64

In [77]:
#데이터 이상치 확인
#to_do

In [78]:
#연금보험료 및 연매출액을 다루기 쉽게 데이터프레임 변경
df=df_ori[['회사ID','년도','월별_연금보험료','연매출액']]
df

,회사ID,년도,월별_연금보험료,연매출액
0,233757,2018,8912380,41688077
1,123464,2018,942380,1760695
2,342124,2018,781180,3221341
3,386420,2015,3795900,4815584
4,129243,2018,40724680,9799736
...,...,...,...,...
99796,171439,2017,2402740,4630622
99797,239388,2018,1428800,1045021
99798,272304,2018,392560,181898
99799,362194,2018,874780,2100218


In [79]:
#연금보험료를 다루기 쉽게 데이터프레임 변경 
#월별 연금보험료의 결측치를 고려해 연도별 월 평균 연금보험료로 바꿈
df_year = df.groupby(['회사ID','년도']).mean()
df_year.reset_index(inplace=True)
df_year.rename(columns = {'월별_연금보험료' : '연도별_월평균연금보험료'}, inplace = True)
df_year

,회사ID,년도,연도별_월평균연금보험료,연매출액
0,63,2018,410128228.33,11328779396.00
1,95,2018,214837843.33,656947437.00
2,102,2018,173769522.50,742658938.00
3,295,2018,77312035.00,83776697.00
4,414,2018,77590193.33,19732769.00
...,...,...,...,...
10106,2400284,2018,1055497.14,1263406.00
10107,2437574,2018,636740.00,214016.00
10108,2693684,2019,438120.00,238076.00
10109,3312354,2019,1312200.00,1451563.00


**2. 연금보험료 측면 데이터 분석** 

**2.1. 기간 내 연금보험료 평균값이 높은 기업 분석**

In [86]:
#기간 내 연금보험료 평균값이 높은 기업 분석
df_year_penrank=df_year[['회사ID','연도별_월평균연금보험료']]
df_year_penrank=df_year_penrank.groupby(['회사ID']).mean()
df_year_penrank.reset_index(inplace=True)
df_year_penrank.sort_values(by='연도별_월평균연금보험료',ascending=False,inplace=True)

In [90]:
#기간 내 연금보험료 최대값이 높은 기업 Top 50개 획득
df_pen_top50=df_year_penrank.head(50)
df_pen_top50.reset_index(inplace=True)
df_pen_top50.drop(['index'],axis=1,inplace=True)
df_pen_top50

,회사ID,연도별_월평균연금보험료
0,227414,848532802.50
1,419945,838814933.33
2,227415,833628100.42
3,126516,564440444.44
4,126521,528434259.58
5,439902,489029400.00
6,469458,482713946.11
7,63,410128228.33
8,419977,391094838.33
9,547586,388365113.33


**2.2. 기간 내 연금보험료 평균 상승율이 높은 기업 분석**

In [23]:
#기업별 기간데이터 추출
df_period=(df_year.sort_values(['년도']).groupby(['회사ID']).년도.max())-(df_year.sort_values(['년도']).groupby(['회사ID']).년도.min())
df_period=pd.DataFrame(df_period).reset_index()
df_period.rename(columns={'년도':'기간'},inplace=True)
df_period

,회사ID,기간
0,63,0
1,95,0
2,102,0
3,295,0
4,414,0
...,...,...
4995,2400284,0
4996,2437574,0
4997,2693684,0
4998,3312354,0


In [24]:
df_period=df_period[df_period['기간']!=0]
df_period

,회사ID,기간
52,14271,4
160,44851,2
217,59533,3
402,122968,3
404,122983,2
...,...,...
4781,515562,1
4784,515837,1
4786,516607,2
4787,516653,2


In [25]:
#기업별 연도별_월평균보험료 변화량 추출-max년도
df_pen_max=(df_year.sort_values(['년도']).groupby(['회사ID']).년도.max())
df_pen_max=pd.DataFrame(df_pen_max).reset_index()
df_pen_max=pd.merge(df_pen_max,df_year)
df_pen_max.rename(columns = {'연도별_월평균연금보험료' : 'max년도_월평균연금보험료'}, inplace = True)
df_pen_max.rename(columns = {'년도' : '년도max'}, inplace = True)
df_pen_max.rename(columns = {'연매출액' : 'max년도_연매출액'}, inplace = True)
df_pen_max

,회사ID,년도max,max년도_월평균연금보험료,max년도_연매출액
0,63,2018,410128228.33,11328779396.00
1,95,2018,214837843.33,656947437.00
2,102,2018,173769522.50,742658938.00
3,295,2018,77312035.00,83776697.00
4,414,2018,77590193.33,19732769.00
...,...,...,...,...
4995,2400284,2018,1055497.14,1263406.00
4996,2437574,2018,636740.00,214016.00
4997,2693684,2019,438120.00,238076.00
4998,3312354,2019,1312200.00,1451563.00


In [26]:
#기업별 연도별_월평균보험료 변화량 추출-min년도
df_pen_min=(df_year.sort_values(['년도']).groupby(['회사ID']).년도.min())
df_pen_min=pd.DataFrame(df_pen_min).reset_index()
df_pen_min=pd.merge(df_pen_min,df_year)
df_pen_min.rename(columns = {'연도별_월평균연금보험료' : 'min년도_월평균연금보험료'}, inplace = True)
df_pen_min.rename(columns = {'년도' : '년도min'}, inplace = True)
df_pen_min.rename(columns = {'연매출액' : 'min년도_연매출액'}, inplace = True)
df_pen_min

,회사ID,년도min,min년도_월평균연금보험료,min년도_연매출액
0,63,2018,410128228.33,11328779396.00
1,95,2018,214837843.33,656947437.00
2,102,2018,173769522.50,742658938.00
3,295,2018,77312035.00,83776697.00
4,414,2018,77590193.33,19732769.00
...,...,...,...,...
4995,2400284,2018,1055497.14,1263406.00
4996,2437574,2018,636740.00,214016.00
4997,2693684,2019,438120.00,238076.00
4998,3312354,2019,1312200.00,1451563.00


In [27]:
#기업별 연도별_월평균보험료 변화량 추출
df_pen_diff=pd.merge(df_pen_min,df_pen_max)
df_pen_diff['변화비율']=df_pen_diff['min년도_월평균연금보험료']/df_pen_diff['max년도_월평균연금보험료']
df_pen_diff=df_pen_diff.drop(['년도min','년도max','max년도_월평균연금보험료','min년도_월평균연금보험료','min년도_연매출액','max년도_연매출액'],axis=1)
df_pen_diff

,회사ID,변화비율
0,63,1.00
1,95,1.00
2,102,1.00
3,295,1.00
4,414,1.00
...,...,...
4995,2400284,1.00
4996,2437574,1.00
4997,2693684,1.00
4998,3312354,1.00


In [28]:
df_pen_dif=pd.merge(df_pen_diff,df_period)
df_pen_dif['기간']=df_pen_dif['기간'].astype(float)
df_pen_dif.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2589 entries, 0 to 2588
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사ID    2589 non-null   int64  
 1   변화비율    2589 non-null   float64
 2   기간      2589 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 80.9 KB


In [29]:
df_pen_dif['CAGR']=(df_pen_dif['변화비율'])**(1/df_pen_dif['기간'])-1
df_pen_cagr=df_pen_dif
df_pen_cagr

,회사ID,변화비율,기간,CAGR
0,14271,0.64,4.00,-0.11
1,44851,1.32,2.00,0.15
2,59533,0.90,3.00,-0.03
3,122968,1.31,3.00,0.09
4,122983,1.55,2.00,0.25
...,...,...,...,...
2584,515562,0.31,1.00,-0.69
2585,515837,1.09,1.00,0.09
2586,516607,1.16,2.00,0.07
2587,516653,0.81,2.00,-0.10


In [34]:
df_pen_cagr_rank=df_pen_cagr.sort_values(by='CAGR',ascending=False)

In [36]:
#기간 내 연금보험료 평균 상승율이 높은 기업 Top 50개 획득
df_pencagr_top50=df_pen_cagr_rank.head(50)
df_pencagr_top50.reset_index(inplace=True)
df_pencagr_top50.drop(['변화비율','기간','index'],axis=1,inplace=True)
df_pencagr_top50

,회사ID,CAGR
0,295827,1.45
1,216628,1.31
2,507862,1.15
3,213842,1.00
4,342869,0.98
5,470208,0.95
6,398665,0.86
7,422744,0.83
8,165510,0.82
9,408352,0.80


**2.3. 기간 내 매출/연금보험료 평균값이 높은 기업 분석**

In [45]:
df_spp=df_year
df_spp['연매출액/연도별_월평균연금보험료']=df_spp['연매출액']/df_spp['연도별_월평균연금보험료']
df_spp

,회사ID,년도,연도별_월평균연금보험료,연매출액,연매출액/연도별_월평균연금보험료
0,63,2018,410128228.33,11328779396.00,27.62
1,95,2018,214837843.33,656947437.00,3.06
2,102,2018,173769522.50,742658938.00,4.27
3,295,2018,77312035.00,83776697.00,1.08
4,414,2018,77590193.33,19732769.00,0.25
...,...,...,...,...,...
10106,2400284,2018,1055497.14,1263406.00,1.20
10107,2437574,2018,636740.00,214016.00,0.34
10108,2693684,2019,438120.00,238076.00,0.54
10109,3312354,2019,1312200.00,1451563.00,1.11


In [92]:
#기간 내 연금보험료 평균값이 높은 기업 분석
df_year_spprank=df_spp[['회사ID','연매출액/연도별_월평균연금보험료']]
df_year_spprank=df_year_spprank.groupby(['회사ID']).mean()
df_year_spprank.reset_index(inplace=True)
df_year_spprank.sort_values(by='연매출액/연도별_월평균연금보험료',ascending=False,inplace=True)
df_year_spprank

,회사ID,연매출액/연도별_월평균연금보험료
4964,602137,inf
2666,288014,inf
4909,567650,225.72
2770,297874,143.73
4148,433492,125.41
...,...,...
363,108773,0.00
4866,552832,0.00
4927,570867,0.00
288,81619,0.00


In [94]:
#기간 내 매출/연금보험료 최대값이 높은 기업 Top 50개 획득
df_spp_top50=df_year_spprank.head(50)
df_spp_top50.reset_index(inplace=True)
df_spp_top50.drop(['index'],axis=1,inplace=True)
df_spp_top50

,회사ID,연매출액/연도별_월평균연금보험료
0,602137,inf
1,288014,inf
2,567650,225.72
3,297874,143.73
4,433492,125.41
5,426180,100.26
6,102971,83.27
7,157237,76.85
8,260890,76.31
9,146654,71.86


**2.4. 기간 내 매출/연금보험료 상승률이 높은 기업 분석**

In [50]:
#기업별 연도별_월평균보험료 변화량 추출-max년도
df_spp_max=(df_year.sort_values(['년도']).groupby(['회사ID']).년도.max())
df_spp_max=pd.DataFrame(df_spp_max).reset_index()
df_spp_max=pd.merge(df_spp_max,df_year)
df_spp_max.drop(['연도별_월평균연금보험료','연매출액'],axis=1,inplace=True)
df_spp_max.rename(columns = {'연매출액/연도별_월평균연금보험료' : 'max년도_연매출액/월평균연금보험료'}, inplace = True)
df_spp_max.rename(columns = {'년도' : '년도max'}, inplace = True)
df_spp_max

,회사ID,년도max,max년도_연매출액/월평균연금보험료
0,63,2018,27.62
1,95,2018,3.06
2,102,2018,4.27
3,295,2018,1.08
4,414,2018,0.25
...,...,...,...
4995,2400284,2018,1.20
4996,2437574,2018,0.34
4997,2693684,2019,0.54
4998,3312354,2019,1.11


In [51]:
#기업별 연도별_월평균보험료 변화량 추출-min년도
df_spp_min=(df_year.sort_values(['년도']).groupby(['회사ID']).년도.min())
df_spp_min=pd.DataFrame(df_spp_min).reset_index()
df_spp_min=pd.merge(df_spp_min,df_year)
df_spp_min.drop(['연도별_월평균연금보험료','연매출액'],axis=1,inplace=True)
df_spp_min.rename(columns = {'연매출액/연도별_월평균연금보험료' : 'min년도_연매출액/월평균연금보험료'}, inplace = True)
df_spp_min.rename(columns = {'년도' : '년도min'}, inplace = True)
df_spp_min

,회사ID,년도min,min년도_연매출액/월평균연금보험료
0,63,2018,27.62
1,95,2018,3.06
2,102,2018,4.27
3,295,2018,1.08
4,414,2018,0.25
...,...,...,...
4995,2400284,2018,1.20
4996,2437574,2018,0.34
4997,2693684,2019,0.54
4998,3312354,2019,1.11


In [52]:
#기업별 연도별_월평균보험료 변화량 추출
df_spp_diff=pd.merge(df_spp_min,df_spp_max)
df_spp_diff['변화비율']=df_spp_diff['min년도_연매출액/월평균연금보험료']/df_spp_diff['max년도_연매출액/월평균연금보험료']
df_spp_diff=df_spp_diff.drop(['년도min','년도max','min년도_연매출액/월평균연금보험료','max년도_연매출액/월평균연금보험료'],axis=1)
df_spp_diff

,회사ID,변화비율
0,63,1.00
1,95,1.00
2,102,1.00
3,295,1.00
4,414,1.00
...,...,...
4995,2400284,1.00
4996,2437574,1.00
4997,2693684,1.00
4998,3312354,1.00


In [53]:
df_spp_dif=pd.merge(df_spp_diff,df_period)
df_spp_dif['기간']=df_spp_dif['기간'].astype(float)
df_spp_dif.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2589 entries, 0 to 2588
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   회사ID    2589 non-null   int64  
 1   변화비율    2589 non-null   float64
 2   기간      2589 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 80.9 KB


In [54]:
df_spp_dif['CAGR']=(df_spp_dif['변화비율'])**(1/df_spp_dif['기간'])-1
df_spp_cagr=df_spp_dif
df_spp_cagr

,회사ID,변화비율,기간,CAGR
0,14271,0.21,4.00,-0.32
1,44851,0.60,2.00,-0.23
2,59533,1.49,3.00,0.14
3,122968,0.91,3.00,-0.03
4,122983,1.64,2.00,0.28
...,...,...,...,...
2584,515562,2.16,1.00,1.16
2585,515837,1.44,1.00,0.44
2586,516607,1.14,2.00,0.07
2587,516653,1.44,2.00,0.20


In [56]:
df_spp_cagr_rank=df_spp_cagr.sort_values(by='CAGR',ascending=False)
df_spp_cagr_rank

,회사ID,변화비율,기간,CAGR
2360,448522,270.20,1.00,269.20
1457,293773,110.88,1.00,109.88
1196,247382,55.80,1.00,54.80
1343,265937,2622.61,3.00,12.79
1207,248815,13.59,1.00,12.59
...,...,...,...,...
2169,417469,0.03,2.00,-0.83
1362,268810,0.00,3.00,-0.86
2240,432907,0.02,2.00,-0.86
1132,242254,0.01,2.00,-0.92


In [57]:
#기간 내 연금보험료 평균 상승율이 높은 기업 Top 50개 획득
df_sppcagr_top50=df_spp_cagr_rank.head(50)
df_sppcagr_top50.reset_index(inplace=True)
df_sppcagr_top50.drop(['변화비율','기간','index'],axis=1,inplace=True)
df_sppcagr_top50

,회사ID,CAGR
0,448522,269.20
1,293773,109.88
2,247382,54.80
3,265937,12.79
4,248815,12.59
5,275496,9.62
6,438349,7.63
7,261744,6.51
8,485596,5.63
9,411214,5.54


**3.최종산출물**